In [1]:
! pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.1 MB/s eta 0:00:00


In [2]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [3]:
!pip install transformers[torch]

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from transformers import pipeline


In [5]:
songs = pd.read_csv("combined_songs.csv")
songs = songs[:10000]

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

pipe = pipeline("text-generation", model="gpt2")

def get_response(prompt: str) -> None:
    """
    Generate a response from the GPT2 model.

    Parameters:
        prompt (str): The user's input/question for the model.

    Returns:
        None: Prints the model's response.
    """
    sequences = pipe(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=256,
        truncation=True,
    )
    print("Chatbot:", sequences[0]['generated_text'])

train_data, test_data = train_test_split(songs, test_size = 0.2, random_state = 69)

train_data_dict = {'lyrics': [str(ele["lyrics"]) if not pd.isnull(ele["lyrics"]) else '' for _, ele in train_data.iterrows()]}
train_data = Dataset.from_dict(train_data_dict)

test_data_dict = {'lyrics': [str(ele["lyrics"]) if not pd.isnull(ele["lyrics"]) else '' for _, ele in test_data.iterrows()]}
test_data = Dataset.from_dict(test_data_dict)

if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token


def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples["lyrics"], max_length=40, padding="max_length", truncation=True)
    # Shift input ids to the right to create labels
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
lm_train_data = train_data.map(tokenize_function, batched=True, num_proc=4, remove_columns=["lyrics"])
lm_test_data = test_data.map(tokenize_function, batched=True, num_proc=4, remove_columns=["lyrics"])

train_valid_data = lm_train_data.train_test_split(test_size=0.2)
train_data = train_valid_data["train"]
valid_data = train_valid_data["test"]

Map (num_proc=4):   0%|          | 0/8000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2000 [00:00<?, ? examples/s]

In [7]:
training_args = TrainingArguments(
    "GPT2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=valid_data,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [8]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.305100,2.974645
2,2.971700,2.919414
3,2.889800,2.905664


TrainOutput(global_step=2400, training_loss=3.03057851155599, metrics={'train_runtime': 456.2782, 'train_samples_per_second': 42.08, 'train_steps_per_second': 5.26, 'total_flos': 391938048000000.0, 'train_loss': 3.03057851155599, 'epoch': 3.0})

In [9]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 18.28


In [10]:
get_response("When i get on the dance floor")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: When i get on the dance floor with this girl who is a supermodel, I'm like, "Whoa!" It's like, you know, this isn't what she wants to do and she has to do this. And you can't be a good dancer.

You know, there are two things that I'm going out to do. One is to do a video on Instagram. It's not about the video. It's about the video. So what I've done is I have a video and a video and an image. So you can be a good dancer and be a good dancer but you'll have to learn how to do it. So that's why I'm in there doing this and I'm doing this.

So that's where the video comes into play.

Yeah, I'm a big, strong girl who's a great dancer. And I don't want you to be a good dancer and you won't be a good dancer. I'll tell ya what, I don't want to be a bad dancer. I'll tell ya what, I want to be great because I'm a big dancer. I'm a girl who wants to be great. But I'm a big, strong girl. I'm just a
